In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

/Users/abohane/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
cost_data = pd.read_csv('data/cost_v1.csv')
dwell_data = pd.read_csv('data/dwell_time_v1.csv')
mall_data = pd.read_csv('data/mall_v1.csv')
mall_traffic_data = pd.read_csv('data/mall_traffic.csv')
people_data = pd.read_csv('data/people_flow_v1.csv')
store_data = pd.read_csv('data/store_v1.csv')
ref_data = pd.read_csv('data/referenciel_traffic_v1.csv')

In [3]:
merge_mall_stores = pd.merge(store_data, mall_data, on='mall_id', how='inner')
merge_first_three = pd.merge(merge_mall_stores, cost_data, how='inner', on='store_id')
cost_df = merge_first_three

positive_cost_cols = ['store_id', 'marketing_costs_r12m', 'service_charges_r12m',
       'administration_fees_r12m', 'real_estate_taxes_r12m','large_scale_works_r12m', 'maintenance_works_r12m']



Index(['store_name', 'store_id', 'mall_id', 'store_year_opened',
       'store_year_closed', 'store_branch_level_1', 'store_branch_level_2',
       'store_branch_level_3', 'store_sales_r12m', 'store_rent_r12m',
       'store_area', 'mall_name', 'mall_country', 'total_mall_area',
       'rent_incentive_r12m', 'service_charges_r12m', 'marketing_costs_r12m',
       'administration_fees_r12m', 'real_estate_taxes_r12m',
       'large_scale_works_r12m', 'maintenance_works_r12m'],
      dtype='object')

In [4]:
positive_cost_cols = ['store_id', 'marketing_costs_r12m', 'service_charges_r12m',
       'administration_fees_r12m', 'real_estate_taxes_r12m','large_scale_works_r12m', 'maintenance_works_r12m']

In [5]:
cost_df['rent_incentive_r12m'] = cost_df['rent_incentive_r12m'].fillna(0)
cost_df['rent_incentive_r12m'] = cost_df['rent_incentive_r12m'].apply(lambda x: 0 if x > 0 else x)

In [6]:
# if values are negative or NaN, replace in cost_df[cost_cols] then replace with mean value for that mall and branch_type_1
for col in positive_cost_cols:
    cost_df[col] = cost_df[col].apply(lambda x: np.nan if x < 0 else x)
    cost_df[col] = cost_df[col].fillna(0)

In [7]:
cost_df.describe()

,store_id,mall_id,store_sales_r12m,store_rent_r12m,store_area,total_mall_area,rent_incentive_r12m,service_charges_r12m,marketing_costs_r12m,administration_fees_r12m,real_estate_taxes_r12m,large_scale_works_r12m,maintenance_works_r12m
count,14111.000000,14111.000000,1.070800e+04,1.028800e+04,14111.000000,14111.000000,1.411100e+04,1.411100e+04,14111.000000,14111.000000,1.411100e+04,14111.000000,1.411100e+04
mean,7752.111544,41.337042,5.667442e+06,6.215413e+05,322.680861,80368.655439,-6.902918e+02,3.332898e+04,5149.153852,1479.567713,1.575449e+04,1314.692651,1.346708e+04
std,4689.895842,27.809628,2.913375e+07,1.746837e+06,1155.034428,39641.142551,2.416321e+04,9.359313e+04,10466.822197,5685.194652,5.182547e+04,10436.976539,5.029620e+04
min,0.000000,0.000000,0.000000e+00,-1.617320e+05,0.000000,941.700000,-2.543159e+06,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00
25%,3562.500000,14.000000,4.763612e+05,8.447000e+04,0.000000,47343.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00
50%,7192.000000,37.000000,1.127216e+06,1.778185e+05,92.900000,73650.100000,0.000000e+00,8.660000e+03,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00
75%,12006.500000,67.000000,3.134927e+06,4.240772e+05,234.685000,104280.100000,0.000000e+00,3.014700e+04,6678.000000,0.000000,1.076850e+04,0.000000,8.285000e+02
max,15550.000000,85.000000,1.615037e+09,6.461468e+07,25677.190000,189072.100000,0.000000e+00,3.071876e+06,218914.000000,213470.000000,1.257501e+06,658168.000000,1.528918e+06


In [8]:
# replace store sales NaN wiith mena values:
cost_df['store_sales_r12m'] = cost_df['store_sales_r12m'].apply(lambda x: np.nan if x <= 1 else x)
cost_df['store_sales_r12m'] = cost_df['store_sales_r12m'].fillna(
    cost_df.groupby(['mall_id', 'store_branch_level_3'])[col].transform('mean'))
cost_df['store_sales_r12m'] = cost_df['store_sales_r12m'].apply(lambda x: np.nan if x <= 1 else x)
cost_df['store_sales_r12m'] = cost_df['store_sales_r12m'].fillna(
    cost_df.groupby(['mall_id', 'store_branch_level_2'])[col].transform('mean'))

cost_df['store_sales_r12m'] = cost_df['store_sales_r12m'].apply(lambda x: np.nan if x <= 1 else x)
cost_df['store_sales_r12m'] = cost_df['store_sales_r12m'].fillna(
    cost_df.groupby(['mall_id', 'store_branch_level_1'])[col].transform('mean'))

cost_df['store_sales_r12m'] = cost_df['store_sales_r12m'].apply(lambda x: np.nan if x <= 1 else x)
cost_df['store_sales_r12m'] = cost_df['store_sales_r12m'].fillna(
    cost_df.groupby(['mall_id'])[col].transform('mean'))



cost_df['store_rent_r12m'] = cost_df['store_rent_r12m'].apply(lambda x: np.nan if x <= 1 else x)
cost_df['store_rent_r12m'] = cost_df['store_rent_r12m'].fillna(
    cost_df.groupby(['mall_id', 'store_branch_level_3'])[col].transform('mean'))

cost_df['store_rent_r12m'] = cost_df['store_rent_r12m'].apply(lambda x: np.nan if x <= 1 else x)
cost_df['store_rent_r12m'] = cost_df['store_rent_r12m'].fillna(
    cost_df.groupby(['mall_id', 'store_branch_level_2'])[col].transform('mean'))

cost_df['store_rent_r12m'] = cost_df['store_rent_r12m'].apply(lambda x: np.nan if x <= 1 else x)

cost_df['store_rent_r12m'] = cost_df['store_rent_r12m'].fillna(
    cost_df.groupby(['mall_id', 'store_branch_level_1'])[col].transform('mean'))

cost_df['store_rent_r12m'] = cost_df['store_rent_r12m'].apply(lambda x: np.nan if x <= 1 else x)
cost_df['store_rent_r12m'] = cost_df['store_rent_r12m'].fillna(
    cost_df.groupby(['mall_id'])[col].transform('mean'))


# store area
cost_df['store_area'] = cost_df['store_area'].apply(lambda x: np.nan if x <= 1 else x)
cost_df['store_area'] = cost_df['store_area'].fillna(
    cost_df.groupby(['mall_id', 'store_branch_level_3'])['store_area'].transform('mean'))

cost_df['store_area'] = cost_df['store_area'].fillna(
    cost_df.groupby(['mall_id', 'store_branch_level_2'])['store_area'].transform('mean'))

cost_df['store_area'] = cost_df['store_area'].fillna(
    cost_df.groupby(['mall_id', 'store_branch_level_1'])['store_area'].transform('mean'))

cost_df['store_area'] = cost_df['store_area'].fillna(
    cost_df.groupby(['mall_id'])['store_area'].transform('mean'))


In [9]:
cost_df['store_area'].isna().sum()

0

In [10]:
cost_df.dropna(subset=['store_area'], inplace=True)

In [12]:
cost_df.describe()

,store_id,mall_id,store_sales_r12m,store_rent_r12m,store_area,total_mall_area,rent_incentive_r12m,service_charges_r12m,marketing_costs_r12m,administration_fees_r12m,real_estate_taxes_r12m,large_scale_works_r12m,maintenance_works_r12m
count,14111.000000,14111.000000,1.411100e+04,1.411100e+04,14111.00000,14111.000000,1.411100e+04,1.411100e+04,14111.000000,14111.000000,1.411100e+04,14111.000000,1.411100e+04
mean,7752.111544,41.337042,4.304592e+06,4.576240e+05,447.63910,80368.655439,-6.902918e+02,3.332898e+04,5149.153852,1479.567713,1.575449e+04,1314.692651,1.346708e+04
std,4689.895842,27.809628,2.549345e+07,1.515644e+06,1243.13858,39641.142551,2.416321e+04,9.359313e+04,10466.822197,5685.194652,5.182547e+04,10436.976539,5.029620e+04
min,0.000000,0.000000,3.000000e+00,3.666667e+00,3.25000,941.700000,-2.543159e+06,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00
25%,3562.500000,14.000000,5.640109e+04,2.283300e+04,90.93750,47343.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00
50%,7192.000000,37.000000,6.636624e+05,1.057790e+05,177.40000,73650.100000,0.000000e+00,8.660000e+03,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00
75%,12006.500000,67.000000,2.145648e+06,2.871250e+05,372.65000,104280.100000,0.000000e+00,3.014700e+04,6678.000000,0.000000,1.076850e+04,0.000000,8.285000e+02
max,15550.000000,85.000000,1.615037e+09,6.461468e+07,25677.19000,189072.100000,0.000000e+00,3.071876e+06,218914.000000,213470.000000,1.257501e+06,658168.000000,1.528918e+06


In [13]:
# total occupancy cost

cost_df['total_occupancy_cost'] = cost_df['store_rent_r12m'] + cost_df['marketing_costs_r12m'] + cost_df['service_charges_r12m'] + cost_df['administration_fees_r12m'] + cost_df['real_estate_taxes_r12m'] + cost_df['large_scale_works_r12m'] + cost_df['maintenance_works_r12m'] + cost_df['rent_incentive_r12m']

# cost to sales ratio:
cost_df['cost_to_sales_ratio'] = cost_df['total_occupancy_cost'] / cost_df['store_sales_r12m']

# cost to area ratio:
cost_df['cost_to_area_ratio'] = cost_df['total_occupancy_cost'] / cost_df['store_area']


In [15]:
cost_df['cost_to_sales_ratio'].describe()

cost_df.to_csv('data/total_cost_df.csv', index=False)

In [16]:

cost_df['store_year_closed'] = pd.to_datetime(cost_df['store_year_closed'], errors='coerce').dt.year
cost_df['store_year_closed'].count()

/var/folders/61/n2sfq6s16zdgx5_xwt6gtdvr0000gn/T/ipykernel_70652/3714940587.py:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  cost_df['store_year_closed'] = pd.to_datetime(cost_df['store_year_closed'], errors='coerce').dt.year


4140

In [17]:
analysis_df = cost_df


Now for the analysis using classifier to predict if a store will close